In [1]:
!pip install libs_qrem
!pip install qiskit-ignis   

In [2]:
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import standard_errors, ReadoutError

def getNoiseModel_no_readouterror(self):
		# Depolarizing quantum errors
		oneQubitError = standard_errors.depolarizing_error(self.oneQubitErrorProb, 1)
		twoQubitError = standard_errors.depolarizing_error(self.twoQubitErrorProb, 2)
		qubitReadoutError = ReadoutError([self.zeroReadoutErrorProb, self.oneReadoutErrorProb])

		# Add errors to noise model
		noiseModel = NoiseModel()
		noiseModel.add_all_qubit_quantum_error(oneQubitError, self.basisGateSetString)
		noiseModel.add_all_qubit_quantum_error(twoQubitError, ['cx'])
		

		return noiseModel

In [6]:
from qiskit import QuantumRegister, Aer, execute
from qiskit.ignis.mitigation.measurement import tensored_meas_cal
import numpy as np
import qiskit.circuit.library as qGate
from qiskit.extensions import UnitaryGate
from qiskit.ignis.mitigation.measurement import TensoredMeasFitter
from qatg import QATG
from qatg import QATGFault
from libs_qrem import LeastNormFilter
from qiskit.tools.visualization import plot_histogram
from qiskit.providers.aer.noise import NoiseModel
from qiskit.providers.aer.noise.errors import standard_errors, ReadoutError




class myCNOTFault(QATGFault):
	def __init__(self):
		super(myCNOTFault, self).__init__(qGate.CXGate, [0, 1], f"gateType: CX, qubits: 0-1")
	def createOriginalGate(self):
		return qGate.CXGate()
	def createFaultyGate(self, faultfreeGate):
		if not isinstance(faultfreeGate, qGate.CXGate):
			raise TypeError("what is this faultfreeGate")
		matrix = qGate.CXGate().to_matrix()
		UF = qGate.UGate(0.05*np.pi, 0.05*np.pi, 0.05*np.pi)
		matrix = np.matmul(np.kron(np.eye(2), UF), matrix)
		matrix = np.matmul(matrix, np.kron(UF, np.eye(2)))
		return UnitaryGate(matrix)

generator = QATG(circuitSize = 2, basisGateSet = [qGate.UGate], circuitInitializedStates = {2: [1, 0, 0, 0]}, minRequiredEffectSize = 2)
configurationList = generator.createTestConfiguration([myCNOTFault()])


for configuration in configurationList:
    print(configuration)
    configuration.circuit.draw('mpl')
   
    n = 2
    qr = configuration.quantumRegister 
    mit_pattern = [[0],[1]]
    meas_calibs, state_labels = tensored_meas_cal(mit_pattern=mit_pattern, qr=qr, circlabel='mcal')
    backend = Aer.get_backend('aer_simulator')
    
    no_anynoise_job = execute(configuration.faultfreeQCKT, backend=backend, shots=200000)
    no_anynoise_results = no_anynoise_job.result()
    # Results without mitigation
    no_anynoise =no_anynoise_results.get_counts()
    print("no_anynoise")
    print(no_anynoise)
    
    no_readouterror_job = execute(meas_calibs, backend=backend, shots=200000,noise_model=getNoiseModel_no_readouterror(configuration))
    no_readouterror_cal_results = no_readouterror_job .result()
    print("no_readouterror")
    print(cal_results.get_counts())
    
    noise_job = execute(meas_calibs, backend=backend, shots=200000, noise_model=configuration.getNoiseModel())
    noise_cal_results = noise_job.result()
    noisy_hist = noise_job.result().get_counts()
    print("noisy_hist")
    print(noisy_hist)
    
    meas_fitter = TensoredMeasFitter(noise_cal_results, mit_pattern=mit_pattern)
    print("calibration matrices")
    print(meas_fitter.cal_matrices)
    
    meas_filter = LeastNormFilter(n, meas_fitter.cal_matrices)
    
   
    
    mitigated_hist = meas_filter.apply(noisy_hist)
    print("mitigated_hist")
    print(mitigated_hist)
    
    
    
    
    job4 = execute(configuration.faultfreeQCKT, backend=backend, shots=200000,noise_model=getNoiseModel_no_readouterror(configuration))
    results4 = job4.result()
    # Results without mitigation
    no_noise = results4.get_counts()
    print("no_readouterror")
    print(no_noise)
    
    
    
    job3 = execute(configuration.faultfreeQCKT, backend=backend, shots=200000, noise_model=configuration.getNoiseModel())
    results3 = job3.result()
    
    # Results without mitigation
    raw_counts = results3.get_counts()

    # Get the filter object
    #meas_filter = meas_fitter.filter
    #print(meas_filter.cal_matrices)
    # Results with mitigation
    mitigated_results = meas_filter.apply(results3)
    mitigated_counts = mitigated_results.get_counts()
    print("raw_counts")
    print(raw_counts)
    print("mitigated_counts")
    print(mitigated_counts)


Target fault: { gateType: CX, qubits: 0-1 }
Length: 10	Repetition: 18	Cost: 180
Chi-Value boundary: 11.472503813225105
Effect Size: 2.1369908626855216
Overkill: 0.0116	Test Escape: 0.0122

no_anynoise
{'11': 135397, '01': 11073, '10': 35946, '00': 17584}
no_readouterror
[{'00': 5000}, {'11': 5000}]
noisy_hist
[{'00': 194095, '01': 2955, '10': 2916, '11': 34}, {'11': 194115, '10': 2929, '01': 2915, '00': 41}]
calibration matrices
[array([[0.985055, 0.01485 ],
       [0.014945, 0.98515 ]]), array([[0.98525, 0.01478],
       [0.01475, 0.98522]])]
mitigated_hist
[{'00': 199985.71476384465, '01': 7.14261807766466, '10': 7.142618077665518}, {'01': 2.0509338082142348, '10': 2.050933808213888, '11': 199995.8981323836}]
no_readouterror
{'00': 30852, '01': 27452, '11': 99669, '10': 42027}
raw_counts
{'01': 28365, '10': 42099, '11': 98550, '00': 30986}
mitigated_counts
{'00': 30872.69749891401, '01': 27238.320873627148, '10': 41395.53447267963, '11': 100493.44715477923}
